In [1]:
import numpy as np
import pandas as pd
import missingno as msno

In [3]:
covid_cases = pd.read_csv('data/covid_cases.csv')
covid_fatalities = pd.read_csv('data/covid_fatalities.csv')

In [5]:
covid_cases.head(20)

,"COVID-19 Total Cases by County, 2020/03/04 - 2021/09/12 at 9:30AM CST",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 549,Unnamed: 550,Unnamed: 551,Unnamed: 552,Unnamed: 553,Unnamed: 554,Unnamed: 555,Unnamed: 556,Unnamed: 557,Unnamed: 558
0,DISCLAIMER: All data are provisional and are s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,County Name,Cases 03-04-2020,Cases 03-05-2020,Cases 03-06-2020,Cases 03-07-2020,Cases 03-08-2020,Cases 03-09-2020,Cases 03-10-2020,Cases 03-11-2020,Cases 03-12-2020,...,Cases 09-03-2021,Cases 09-04-2021,Cases 09-05-2021,Cases 09-06-2021,Cases 09-07-2021,Cases 09-08-2021,Cases 09-09-2021,Cases 09-10-2021,Cases 09-11-2021,Cases 09-12-2021
2,Anderson,0,0,0,0,0,0,0,0,0,...,4866,4866,4866,4866,4866,4878,4837,4894,4889,4889
3,Andrews,0,0,0,0,0,0,0,0,0,...,2069,2280,2280,2280,2280,2280,2280,2280,2280,2280
4,Angelina,0,0,0,0,0,0,0,0,0,...,5955,5955,5955,5955,5955,5955,6086,6086,6086,6086
5,Aransas,0,0,0,0,0,0,0,0,0,...,1587,1616,1629,1629,1629,1713,1740,1759,1775,1780
6,Archer,0,0,0,0,0,0,0,0,0,...,848,853,853,853,853,866,871,871,874,874
7,Armstrong,0,0,0,0,0,0,0,0,0,...,173,173,173,173,173,174,176,175,175,175
8,Atascosa,0,0,0,0,0,0,0,0,0,...,5772,5800,5800,5800,5800,5967,6002,6012,6012,6012
9,Austin,0,0,0,0,0,0,0,0,0,...,2301,2312,2324,2339,2349,2373,2392,2404,2411,2416


In [9]:
covid_cases.columns = covid_cases.iloc[1]
covid_cases = covid_cases.drop(index=[0,1])
covid_cases

1,County Name,Cases 03-04-2020,Cases 03-05-2020,Cases 03-06-2020,Cases 03-07-2020,Cases 03-08-2020,Cases 03-09-2020,Cases 03-10-2020,Cases 03-11-2020,Cases 03-12-2020,...,Cases 09-03-2021,Cases 09-04-2021,Cases 09-05-2021,Cases 09-06-2021,Cases 09-07-2021,Cases 09-08-2021,Cases 09-09-2021,Cases 09-10-2021,Cases 09-11-2021,Cases 09-12-2021
2,Anderson,0,0,0,0,0,0,0,0,0,...,4866,4866,4866,4866,4866,4878,4837,4894,4889,4889
3,Andrews,0,0,0,0,0,0,0,0,0,...,2069,2280,2280,2280,2280,2280,2280,2280,2280,2280
4,Angelina,0,0,0,0,0,0,0,0,0,...,5955,5955,5955,5955,5955,5955,6086,6086,6086,6086
5,Aransas,0,0,0,0,0,0,0,0,0,...,1587,1616,1629,1629,1629,1713,1740,1759,1775,1780
6,Archer,0,0,0,0,0,0,0,0,0,...,848,853,853,853,853,866,871,871,874,874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,Notes:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,Case counts do not include probable cases,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,County-level case counts were not available on...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
covid_cases = covid_cases.drop(index=[256,257,258,259,260,261])
covid_cases

1,County Name,Cases 03-04-2020,Cases 03-05-2020,Cases 03-06-2020,Cases 03-07-2020,Cases 03-08-2020,Cases 03-09-2020,Cases 03-10-2020,Cases 03-11-2020,Cases 03-12-2020,...,Cases 09-03-2021,Cases 09-04-2021,Cases 09-05-2021,Cases 09-06-2021,Cases 09-07-2021,Cases 09-08-2021,Cases 09-09-2021,Cases 09-10-2021,Cases 09-11-2021,Cases 09-12-2021
2,Anderson,0,0,0,0,0,0,0,0,0,...,4866,4866,4866,4866,4866,4878,4837,4894,4889,4889
3,Andrews,0,0,0,0,0,0,0,0,0,...,2069,2280,2280,2280,2280,2280,2280,2280,2280,2280
4,Angelina,0,0,0,0,0,0,0,0,0,...,5955,5955,5955,5955,5955,5955,6086,6086,6086,6086
5,Aransas,0,0,0,0,0,0,0,0,0,...,1587,1616,1629,1629,1629,1713,1740,1759,1775,1780
6,Archer,0,0,0,0,0,0,0,0,0,...,848,853,853,853,853,866,871,871,874,874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Wood,0,0,0,0,0,0,0,0,0,...,2810,2810,2810,2810,2810,2934,2934,2982,2982,2982
252,Yoakum,0,0,0,0,0,0,0,0,0,...,1049,1049,1049,1049,1049,1054,1054,1063,1063,1063
253,Young,0,0,0,0,0,0,0,0,0,...,2187,2196,2196,2196,2196,2215,2217,2224,2233,2233
254,Zapata,0,0,0,0,0,0,0,0,0,...,1644,1645,1645,1645,1645,1646,1656,1657,1657,1657
